# Taxi Routes in New York City - 2022

## Contents

Background

What data will be Used

Cleaning the data

  Identifying the taxi zones
    
    tlc_green_trips_2022
        
    taxi_zone_geom

  Identifying the boroughs
    
Analysis

Conclusion

References

## Background

New York City is a bustling metropolis, with citizens and tourists alike traveling everywhere within the city. This project tries to gain some insight into different combinations of routes and where a lot of the green taxis go.

## What data will be used

The dataset that will be used is the `new_york_taxi_trips` dataset from the bigquery-public-data database. The data will be queried through BigQuery.

These are the tables and columns that will be used for this analysis:

**tlc_green_trips_2022:**

`pickup_location_id` - the zone number that the passenger(s) started their trip

`dropoff_location_id` - the zone number that the passenger(s) ended their trip

**taxi_zone_geom:**

`zone_id` - the number for every taxi zone (this is the same value as pickup_location_id and dropoff_location_id, so it can be joined between tables.)

`zone_name` - the name of every taxi zone

`borough` - the borough that the taxi zone is in


## Cleaning the Data


### Identifying the Taxi Zones 

These are the four columns of “taxi zones” that will be checked:

**tlc_green_trips_2022 table:**

`pickup_location_id`
`dropoff_location_id`

**taxi_zone_geom table:**

`zone_id`
`zone_name`


**tlc_green_trips_2022**

Checking for values in the `pickup_location_id` column:

In [ ]:
SELECT
  DISTINCT CAST(pickup_location_id AS INT64) AS pickup_number
FROM `bigquery-public-data.new_york_taxi_trips.tlc_green_trips_2022`
ORDER BY pickup_number

Checking for values in the `dropoff_location_id` column:

In [ ]:
SELECT
  DISTINCT CAST(dropoff_location_id AS INT64) AS dropoff_number
FROM `bigquery-public-data.new_york_taxi_trips.tlc_green_trips_2022`
ORDER BY dropoff_number

The `pickup_location_id` column delivers only 258 distinct values between 1 and 265. Because of this, some possible values are not in this table for some reason. The `dropoff_location_id` column returns only 259 distinct values.

#### These are the values that were not in pickup_location_id:

12 (Battery Park), 103 - 105 (Liberty, Ellis, and Governor’s Islands), 110 (Great Kills Park), 199 (Rikers Island), 204 (Rossville/Woodrow)

#### These are the values that were not in dropoff_location_id:

99 (Fresh Kills Park), 103 - 105 (Liberty, Ellis, and Governor’s Islands), 199 (Rikers Island), 204 (Rossville/Woodrow)

Because almost all of these locations are in parks or islands, it is more likely that these were never pickup or dropoff zones in 2022, rather than data actually missing. See the references at the end for a list and map of all of the taxi zones in NYC.


Now, let’s check the `taxi_zone_geom` table.

**taxi_zone_geom**

Checking for values in the `zone_id` column:

In [ ]:
SELECT
DISTINCT CAST(zone_id AS INT64) AS zone_number
FROM `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom`
ORDER BY zone_number

Checking for values in the zone_name column:

SELECT
DISTINCT zone_name
FROM `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom`
ORDER BY zone_name

Both the `zone_id` and `zone_name` columns deliver 260 distinct values ranging from 1-263. Because of this, some possible values are not in this table. The possible zones missing from the table are 57, 104, and 105. Looking into this further, I found out that each of these zones share the same name as other zones. Zones 56 and 57 are both Corona in the Queens borough, and 103, 104, and 105 make up Liberty Island, Ellis Island, and the Governor’s Island, respectively (see references at end). Even though each of these individual islands have a unique zone number, the name of the zone consists of all three islands.

Double checking to see if these zones exist in the table using `WHERE` instead of `DISTINCT`, in case the same zone_name might have hid the other zones.

In [ ]:
SELECT CAST(zone_id AS INT64)
FROM `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom`
WHERE zone_id = "57"

Zone 57 does not appear anywhere.

In [ ]:
SELECT CAST(zone_id AS INT64)
FROM `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom`
WHERE zone_id = "104"

Zone 104 does not appear anywhere.

In [ ]:
SELECT CAST(zone_id AS INT64)
FROM `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom`
WHERE zone_id = "105"

Zone 105 also does not appear anywhere. It could be very likely that these zones were combined together. While I don’t know for sure, I will keep going from here, acknowledging that there might be possible limitations of the data.


### Identifying the Boroughs

I checked for values in the borough column of the `taxi_zone_geom` table:

In [ ]:
SELECT
DISTINCT borough
FROM `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom`
ORDER BY borough

The query delivers six results for boroughs, one of them labeled “EWR”.
This next query finds what rows have EWR listed for the borough:

In [ ]:
SELECT *
FROM `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom`
WHERE borough = "EWR"
LIMIT 1000

The query returns the Newark Airport, which is in Newark, New Jersey, not New York City. Upon further research, the Newark Airport is indeed zone 1 in the NYC taxi system, even though it is not located in NYC.

## Analysis

In this next query, I joined the `zone_id` column from the `taxi_zone_geom` table with the `pickup_location_id` and `dropoff_location_id` columns from the `tlc_green_trips_2022` table. I did this twice, so I could have one column listing the routes between specific taxi zones, and another column listing the routes between the boroughs. I used aliases for all four join statements so I could join columns from the same tables four times.

Then, I aggregated the `pickup_location_id` with `COUNT` to find out which trips were taken the most often in 2022. I am only doing a `COUNT` of pickups because the `COUNT` of pickups and dropoffs is the same, creating a redundant column. To get a percentage for each route out of all the different routes, I used the `OVER` function.

By grouping the pickup and dropoff zones together, I was able to calculate the frequency of every possible combination of pickup and dropoff taxi zones in NYC.

In [ ]:
SELECT
--making the columns taxi_route (zone to zone), borough_route (borough to borough), frequency(a COUNT aggregate of pickup_location_id), and percent (using the OVER function).
  
  CONCAT(geom1.zone_name, " - ", geom2.zone_name) AS taxi_route,
  CONCAT(geom3.borough, " - ", geom4.borough) AS borough_route,
  COUNT(pickup_location_id) AS frequency,
  COUNT(*) * 100.0 / SUM(COUNT(*)) OVER() AS percent
  
FROM `bigquery-public-data.new_york_taxi_trips.tlc_green_trips_2022`

--join the taxi zone names to the ids
--joining pickup_location_id and dropoff_location_id to zone_id of the taxi_zone_geom table (aliased as "geom1" and "geom2") for the taxi_route column
JOIN
  bigquery-public-data.new_york_taxi_trips.taxi_zone_geom AS geom1
ON pickup_location_id = geom1.zone_id
JOIN
  bigquery-public-data.new_york_taxi_trips.taxi_zone_geom AS geom2
ON dropoff_location_id = geom2.zone_id
--joining the same columns to zone_id of the taxi_zone_geom table (aliased as "geom3" and "geom4") for the taxi_borough column
JOIN
  bigquery-public-data.new_york_taxi_trips.taxi_zone_geom AS geom3
ON pickup_location_id = geom3.zone_id
JOIN
  bigquery-public-data.new_york_taxi_trips.taxi_zone_geom AS geom4
ON dropoff_location_id = geom4.zone_id

--grouping by all of the columns that were previously used. Because they were all used, they must be included in the GROUP BY clause.
GROUP BY
  pickup_location_id,
  dropoff_location_id,
  geom1.zone_name,
  geom2.zone_name,
  geom3.borough,
  geom4.borough
--sorting by the COUNT aggregate of pickup_location_id (in descending order) as we want to see which routes were taken the most.
ORDER BY
  COUNT(pickup_location_id) DESC
LIMIT 1000

## Conclusion

Looking at the results, it is not even a competition in that the East Harlem area receives the most amount of routes by the green taxi service, both with pickups and dropoffs. The top two routes listed, between the north and south parts of East Harlem, make up more than 4.2% of all possible route combinations between zones. Taking a bigger picture look, the `borough_route` column shows that most of the high frequent routes take place within Manhattan.

One limitation of this study is that I only looked at a table of trips for green taxis in the year 2022. It is very plausible that tables containing data about the yellow taxis, or tables of the green taxis from previous years, could contain different information and insights.

## References

This is list of all NYC Taxi Zones, made by user fivethirtyeight:
https://github.com/fivethirtyeight/uber-tlc-foil-response/blob/master/uber-trip-data/taxi-zone-lookup.csv

This is a map of all NYC Taxi Zones by cityofnewyork.us:
https://data.cityofnewyork.us/Transportation/NYC-Taxi-Zones/d3c5-ddgc